In [1]:
import os
import re
from datetime import datetime
from tqdm import tqdm
import numpy as np
import pandas as pd

In [2]:
covid_repo_path = "..\\..\\COVID-19"
db_source = os.path.join(covid_repo_path, "csse_covid_19_data\\csse_covid_19_daily_reports")
print(f"list of files: {len(os.listdir(db_source))}")

list of files: 48


In [3]:
data_df = pd.DataFrame()
for file in tqdm(os.listdir(db_source)):
    try:
        crt_date, crt_ext = crt_file = file.split(".")
        if(crt_ext == "csv"):
            crt_date_df = pd.read_csv(os.path.join(db_source, file))
            crt_date_df['date_str'] = crt_date
            crt_date_df['date'] = crt_date_df['date_str'].apply(lambda x: datetime.strptime(x, "%m-%d-%Y"))
            data_df = data_df.append(crt_date_df)
    except:
        pass

 77%|███████▋  | 37/48 [00:00<00:00, 173.57it/s]C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
100%|██████████| 48/48 [00:00<00:00, 156.74it/s]


In [4]:
print(f"Data: rows: {data_df.shape[0]}, cols: {data_df.shape[1]}")
print(f"Days: {data_df.date_str.nunique()} ({data_df.date_str.min()} : {data_df.date_str.max()})")
print(f"Country/Region: {data_df['Country/Region'].nunique()}")
print(f"Province/State: {data_df['Province/State'].nunique()}")
print(f"Confirmed all: {sum(data_df.groupby(['Province/State'])['Confirmed'].max())}")
print(f"Recovered all: {sum(data_df.loc[~data_df.Recovered.isna()].groupby(['Province/State'])['Recovered'].max())}")
print(f"Deaths all: {sum(data_df.loc[~data_df.Deaths.isna()].groupby(['Province/State'])['Deaths'].max())}")

Data: rows: 3992, cols: 10
Days: 46 (01-22-2020 : 03-07-2020)
Country/Region: 105
Province/State: 155
Confirmed all: 82229.0
Recovered all: 55634.0
Deaths all: 3099.0


In [5]:
data_df.head()

,Confirmed,Country/Region,Deaths,Last Update,Latitude,Longitude,Province/State,Recovered,date,date_str
0,1.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Anhui,NaN,2020-01-22,01-22-2020
1,14.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Beijing,NaN,2020-01-22,01-22-2020
2,6.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Chongqing,NaN,2020-01-22,01-22-2020
3,1.0,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Fujian,NaN,2020-01-22,01-22-2020
4,NaN,Mainland China,NaN,1/22/2020 17:00,NaN,NaN,Gansu,NaN,2020-01-22,01-22-2020


In [6]:
data_df.tail()

,Confirmed,Country/Region,Deaths,Last Update,Latitude,Longitude,Province/State,Recovered,date,date_str
220,0.0,Australia,0.0,2020-03-06T04:33:03,-12.4634,130.8456,Northern Territory,0.0,2020-03-07,03-07-2020
221,0.0,US,0.0,2020-02-24T23:33:02,29.3829,-98.6134,"Lackland, TX (From Diamond Princess)",0.0,2020-03-07,03-07-2020
222,0.0,US,0.0,2020-03-07T19:53:02,30.3213,-95.4778,"Montgomery County, TX",0.0,2020-03-07,03-07-2020
223,0.0,US,0.0,2020-02-24T23:33:02,41.2545,-95.9758,"Omaha, NE (From Diamond Princess)",0.0,2020-03-07,03-07-2020
224,0.0,US,0.0,2020-02-24T23:33:02,38.2721,-121.9399,"Travis, CA (From Diamond Princess)",0.0,2020-03-07,03-07-2020


In [7]:
province_state = data_df['Province/State'].unique()

for ps in province_state:

    data_df.loc[(data_df['Province/State']==ps) & (data_df['Latitude'].isna()), 'Latitude'] =\
                data_df.loc[(~data_df['Latitude'].isna()) & \
                            (data_df['Province/State']==ps), 'Latitude'].median()
    
    data_df.loc[(data_df['Province/State']==ps) & (data_df['Longitude'].isna()), 'Longitude'] =\
            data_df.loc[(~data_df['Longitude'].isna()) & \
                        (data_df['Province/State']==ps), 'Longitude'].median() 

In [8]:
country_region = data_df['Country/Region'].unique()

for cr in country_region:

    data_df.loc[(data_df['Country/Region']==cr) & (data_df['Latitude'].isna()), 'Latitude'] =\
                data_df.loc[(~data_df['Latitude'].isna()) & \
                            (data_df['Country/Region']==cr), 'Latitude'].median()
    
    data_df.loc[(data_df['Country/Region']==cr) & (data_df['Longitude'].isna()), 'Longitude'] =\
            data_df.loc[(~data_df['Longitude'].isna()) & \
                        (data_df['Country/Region']==cr), 'Longitude'].median() 

In [9]:
data_df.loc[data_df.Latitude.isna()]

,Confirmed,Country/Region,Deaths,Last Update,Latitude,Longitude,Province/State,Recovered,date,date_str
48,1.0,Ivory Coast,NaN,1/27/20 23:59,NaN,NaN,NaN,NaN,2020-01-27,01-27-2020
78,1.0,Azerbaijan,0.0,2020-02-28T15:03:26,NaN,NaN,NaN,0.0,2020-02-28,02-28-2020
99,1.0,North Ireland,0.0,2020-02-28T05:43:02,NaN,NaN,NaN,0.0,2020-02-28,02-28-2020


In [10]:
data_df.loc[data_df.Longitude.isna()]

,Confirmed,Country/Region,Deaths,Last Update,Latitude,Longitude,Province/State,Recovered,date,date_str
48,1.0,Ivory Coast,NaN,1/27/20 23:59,NaN,NaN,NaN,NaN,2020-01-27,01-27-2020
78,1.0,Azerbaijan,0.0,2020-02-28T15:03:26,NaN,NaN,NaN,0.0,2020-02-28,02-28-2020
99,1.0,North Ireland,0.0,2020-02-28T05:43:02,NaN,NaN,NaN,0.0,2020-02-28,02-28-2020


In [11]:
data_df.to_csv("covid-19-all.csv", index=False)